In [78]:
import pandas as pd
import nltk, re, pprint,random
from nltk.corpus import stopwords
from nltk.util import ngrams
from nltk.probability import FreqDist
from nltk.classify import SklearnClassifier
from nltk import pos_tag, word_tokenize, NaiveBayesClassifier, classify, bigrams
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string

/Users/daniellediehl/anaconda/lib/python2.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [80]:
airline_data = pd.read_csv('airline.csv', encoding = 'latin1')

In [81]:
airline = airline_data[['airline_name','content']]

In [4]:
content_air=airline['airline_name']+' '+ airline['content']
content_air
airline['content2']=content_air

/Users/daniellediehl/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
top_10_airlines_vc=airline['airline_name'].value_counts()[:10]
top_10_airlines=list(top_10_airlines_vc.index.values)
top_10_airlines

[u'spirit-airlines',
 u'british-airways',
 u'united-airlines',
 u'jet-airways',
 u'air-canada-rouge',
 u'emirates',
 u'ryanair',
 u'american-airlines',
 u'lufthansa',
 u'qantas-airways']

In [6]:
#count number of times top 10 airlines occur in content2
#just value counts for top 10
top_10_airlines_vc

spirit-airlines      990
british-airways      901
united-airlines      840
jet-airways          727
air-canada-rouge     715
emirates             691
ryanair              658
american-airlines    612
lufthansa            600
qantas-airways       580
Name: airline_name, dtype: int64

In [7]:
#lift of top 10 brands occurring together
def lift(a, b):
    num = len(airline)
    num_a = len(airline[airline['content2'].str.contains(a)])
    num_b = len(airline[airline['content2'].str.contains(b)])
    if_has_a = airline[airline['content2'].str.contains(a)]
    num_a_and_b = len(if_has_a['content2'][if_has_a['content2'].str.contains(b)])
    return num*float(num_a_and_b)/(num_a * num_b)

In [8]:
lift_df = pd.DataFrame(columns=top_10_airlines, index=top_10_airlines)

for brand1, series in list(lift_df.iterrows()):
    for brand2 in series.index:
        if brand1 != brand2:
            lift_df[brand1].loc[brand2] = lift(brand2, brand1)

lift_df
#here, we see that it is not common to compare airlines in reviews. 
#therefore, when dealing with sentiment, we do not need to worry about proximity

,spirit-airlines,british-airways,united-airlines,jet-airways,air-canada-rouge,emirates,ryanair,american-airlines,lufthansa,qantas-airways
spirit-airlines,NaN,0,0,0,0,0,0,0,0,0
british-airways,0,NaN,0,0,0,0,0,0,0,0
united-airlines,0,0,NaN,0,0,0,0,0,0,0
jet-airways,0,0,0,NaN,0,0.0813441,0,0,0,0
air-canada-rouge,0,0,0,0,NaN,0,0,0,0,0
emirates,0,0,0,0.0813441,0,NaN,0,0,0,0.203921
ryanair,0,0,0,0,0,0,NaN,0,0,0
american-airlines,0,0,0,0,0,0,0,NaN,0,0
lufthansa,0,0,0,0,0,0,0,0,NaN,0
qantas-airways,0,0,0,0,0,0.203921,0,0,0,NaN


In [104]:
#calculate word frequency analysis for content

stop = stopwords.words('english')
airline_data['content_clean'] = airline_data['content'].str.lower()
airline_data['content_clean'] = airline_data['content_clean'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
airline_data[:1]

,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,...,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended,content_clean
0,adria-airways,/airline-reviews/adria-airways,Adria Airways customer review,D Ito,Germany,2015-04-10,Outbound flight FRA/PRN A319. 2 hours 10 min f...,NaN,NaN,Economy,...,7.0,4.0,4.0,4.0,0.0,NaN,NaN,4.0,1,outbound flight fra/prn a319. 2 hours 10 min f...


In [10]:
small=airline_data[:100]

In [11]:
def content_clean_words(df):
    content_list=df.content_clean.values.tolist()
    words=[]
    for i in range(0,len(content_list)):
        words1=word_tokenize(content_list[i])
        words.append(words1)
    return words

In [12]:
def combine_content_words(words):
    content_words = [item for sublist in words for item in sublist]
    return content_words

In [13]:
def lemmatizer(content_words):
    content_words_no_punc = [w for w in content_words if w.isalpha()]
    porter=nltk.PorterStemmer()
    filtered_words= [porter.stem(t) for t in content_words_no_punc] #lemmatize
    return filtered_words

In [14]:
def top_words(filtered_words):
    freqdist = FreqDist(filtered_words)
    top_100=freqdist.most_common(100)
    return top_100

In [15]:
def super_function(df):
    top=top_words(lemmatizer(combine_content_words(content_clean_words(df))))
    return top

In [16]:
super_function(small)

[(u'flight', 202),
 (u'time', 64),
 (u'crew', 57),
 (u'seat', 56),
 (u'servic', 50),
 (u'good', 47),
 (u'friendli', 41),
 (u'airlin', 41),
 (u'cabin', 39),
 (u'return', 36),
 (u'meal', 33),
 (u'food', 31),
 (u'class', 29),
 (u'short', 28),
 (u'serv', 28),
 (u'clean', 28),
 (u'busi', 27),
 (u'board', 27),
 (u'adria', 27),
 (u'leg', 26),
 (u'would', 26),
 (u'fli', 26),
 (u'comfort', 26),
 (u'nice', 26),
 (u'staff', 25),
 (u'drink', 25),
 (u'athen', 25),
 (u'ljubljana', 24),
 (u'new', 23),
 (u'hour', 23),
 (u'plane', 23),
 (u'small', 22),
 (u'airport', 22),
 (u'passeng', 22),
 (u'one', 20),
 (u'aircraft', 20),
 (u'arriv', 19),
 (u'even', 18),
 (u'free', 18),
 (u'pleasant', 18),
 (u'quit', 18),
 (u'minut', 17),
 (u'aegean', 17),
 (u'experi', 17),
 (u'offer', 17),
 (u'flew', 16),
 (u'effici', 16),
 (u'entertain', 15),
 (u'economi', 15),
 (u'inflight', 15),
 (u'excel', 15),
 (u'great', 14),
 (u'room', 14),
 (u'hot', 14),
 (u'first', 14),
 (u'crj', 14),
 (u'though', 14),
 (u'delay', 13),
 (u'

In [17]:
top_5_origin=airline_data['author_country'].value_counts()[:5]
top_5_origin

United Kingdom    9969
United States     8507
Australia         5062
Canada            3303
Germany           1117
Name: author_country, dtype: int64

In [18]:
df_uk=airline_data[airline_data['author_country']=='United Kingdom']
df_us=airline_data[airline_data['author_country']=='United States']
df_aus=airline_data[airline_data['author_country']=='Australia']
df_can=airline_data[airline_data['author_country']=='Canada']
df_ger=airline_data[airline_data['author_country']=='Germany']

In [19]:
top_uk_words=super_function(df_uk)
top_uk_words

[(u'flight', 19210),
 (u'seat', 10009),
 (u'time', 6452),
 (u'good', 5532),
 (u'servic', 5304),
 (u'food', 5222),
 (u'crew', 5070),
 (u'return', 5059),
 (u'staff', 4602),
 (u'airlin', 4453),
 (u'hour', 4447),
 (u'cabin', 4313),
 (u'would', 4249),
 (u'plane', 3845),
 (u'board', 3489),
 (u'fli', 3285),
 (u'one', 3056),
 (u'back', 2873),
 (u'get', 2829),
 (u'drink', 2811),
 (u'delay', 2802),
 (u'passeng', 2787),
 (u'use', 2785),
 (u'check', 2764),
 (u'leg', 2666),
 (u'us', 2600),
 (u'arriv', 2600),
 (u'travel', 2572),
 (u'flew', 2458),
 (u'airport', 2443),
 (u'class', 2416),
 (u'comfort', 2373),
 (u'aircraft', 2365),
 (u'could', 2348),
 (u'meal', 2308),
 (u'book', 2298),
 (u'busi', 2261),
 (u'economi', 2181),
 (u'entertain', 2181),
 (u'first', 2083),
 (u'excel', 2005),
 (u'way', 1935),
 (u'friendli', 1923),
 (u'even', 1896),
 (u'experi', 1851),
 (u'offer', 1805),
 (u'london', 1798),
 (u'new', 1740),
 (u'minut', 1725),
 (u'great', 1697),
 (u'air', 1692),
 (u'go', 1613),
 (u'much', 1609),
 

In [20]:
top_us_words=super_function(df_us)
top_aus_words=super_function(df_aus)
top_can_words=super_function(df_can)
top_ger_words=super_function(df_ger)

In [21]:
#normalize the values-- divide by total num words in that country's reviews (len(content_words))
#build a df with rows=words, col=countries
content_words_us=len(combine_content_words(content_clean_words(df_us)))
content_words_uk=len(combine_content_words(content_clean_words(df_uk)))
content_words_aus=len(combine_content_words(content_clean_words(df_aus)))
content_words_can=len(combine_content_words(content_clean_words(df_can)))
content_words_ger=len(combine_content_words(content_clean_words(df_ger)))

In [22]:
content_words_uk

764399

In [23]:
def top_words_to_list(a):
    top_words_list=[]
    for tup in range(len(a)):
        word=a[tup][0]
        top_words_list.append(word)
    return top_words_list

In [24]:
list_uk_words = top_words_to_list(top_uk_words)
list_us_words = top_words_to_list(top_us_words)
list_aus_words = top_words_to_list(top_aus_words)
list_can_words = top_words_to_list(top_can_words)
list_ger_words = top_words_to_list(top_ger_words)

In [25]:
def top_word_counts(a):
    count_words_list=[]
    for tup in range(len(a)):
        word=a[tup][1]
        count_words_list.append(word)
    return count_words_list

In [26]:
list_uk_counts = top_word_counts(top_uk_words)
list_us_counts = top_word_counts(top_us_words)
list_aus_counts = top_word_counts(top_aus_words)
list_can_counts = top_word_counts(top_can_words)
list_ger_counts = top_word_counts(top_ger_words)

In [27]:
df_uk_ = pd.DataFrame(index=list_uk_words,columns=['UK Word Counts'])
df_uk_['UK Word Counts']=list_uk_counts
df_us_ = pd.DataFrame(index=list_us_words,columns=['US Word Counts'])
df_us_['US Word Counts']=list_us_counts
df_aus_ = pd.DataFrame(index=list_aus_words,columns=['AUS Word Counts'])
df_aus_['AUS Word Counts']=list_aus_counts
df_can_ = pd.DataFrame(index=list_can_words,columns=['CAN Word Counts'])
df_can_['CAN Word Counts']=list_can_counts
df_ger_ = pd.DataFrame(index=list_ger_words,columns=['GER Word Counts'])
df_ger_['GER Word Counts']=list_ger_counts

In [28]:
#5 df then join each one later. full join 
df2=df_uk_.join(df_us_,how='outer')
df3=df2.join(df_aus_, how='outer')
df4=df3.join(df_can_,how='outer')
df_full=df4.join(df_ger_,how='outer')
df_full=df_full.fillna(0)
df_full['US Word Counts']=df_full['US Word Counts']/content_words_us
df_full['UK Word Counts']=df_full['UK Word Counts']/content_words_uk
df_full['AUS Word Counts']=df_full['AUS Word Counts']/content_words_aus
df_full['CAN Word Counts']=df_full['CAN Word Counts']/content_words_can
df_full['GER Word Counts']=df_full['GER Word Counts']/content_words_ger
df_full

,UK Word Counts,US Word Counts,AUS Word Counts,CAN Word Counts,GER Word Counts
ac,0.000000,0.000000,0.000000,0.002348,0.000000
again,0.002053,0.002245,0.002084,0.002311,0.001536
air,0.002214,0.002071,0.002558,0.006748,0.002503
aircraft,0.003094,0.000000,0.002755,0.001636,0.002636
airlin,0.005825,0.008038,0.006628,0.006179,0.005457
airport,0.003196,0.003831,0.003003,0.003614,0.003258
also,0.001733,0.001680,0.001876,0.001502,0.002199
alway,0.000000,0.000000,0.000000,0.000000,0.001603
anoth,0.000000,0.002358,0.000000,0.001884,0.000000
arriv,0.003401,0.003509,0.003514,0.003460,0.003046


In [29]:
top_us_word_freq = df_full['US Word Counts'].where(df_full.T.idxmax().values == 'US Word Counts').dropna().sort_values(ascending = False)
top_uk_word_freq = df_full['UK Word Counts'].where(df_full.T.idxmax().values == 'UK Word Counts').dropna().sort_values(ascending = False)
top_aus_word_freq = df_full['AUS Word Counts'].where(df_full.T.idxmax().values == 'AUS Word Counts').dropna().sort_values(ascending = False)
top_can_word_freq = df_full['CAN Word Counts'].where(df_full.T.idxmax().values == 'CAN Word Counts').dropna().sort_values(ascending = False)
top_ger_word_freq = df_full['GER Word Counts'].where(df_full.T.idxmax().values == 'GER Word Counts').dropna().sort_values(ascending = False)

In [30]:
top_uk_word_freq
top_us_word_freq

flight     0.029906
airlin     0.008038
hour       0.007966
plane      0.006528
us         0.006016
fli        0.005965
delay      0.005209
get        0.005203
board      0.004785
check      0.003855
airport    0.003831
first      0.003769
attend     0.003549
bag        0.003437
could      0.003355
trip       0.003180
gate       0.003092
told       0.002907
minut      0.002829
custom     0.002786
go         0.002729
wait       0.002726
never      0.002686
connect    0.002670
got        0.002620
pay        0.002607
ticket     0.002552
day        0.002542
anoth      0.002358
next       0.002117
i          0.002063
make       0.002059
cancel     0.002045
late       0.001970
it         0.001938
take       0.001927
did        0.001862
spirit     0.001851
two        0.001792
line       0.001765
do         0.001737
made       0.001726
charg      0.001716
chang      0.001687
final      0.001635
unit       0.001585
need       0.001581
said       0.001548
tri        0.001543
took       0.001529


In [31]:
top_aus_word_freq

food         0.007702
staff        0.006462
busi         0.005624
meal         0.004202
comfort      0.003936
entertain    0.003622
flew         0.003564
arriv        0.003514
sydney       0.003501
great        0.003090
excel        0.003048
qanta        0.002611
loung        0.002463
melbourn     0.002434
singapor     0.001741
overal       0.001689
littl        0.001544
Name: AUS Word Counts, dtype: float64

In [32]:
top_can_word_freq

air        0.006748
would      0.005591
canada     0.005175
toronto    0.004569
roug       0.004453
back       0.004355
book       0.003097
way        0.002835
experi     0.002621
ac         0.002348
again      0.002311
room       0.002138
peopl      0.002037
ask        0.001936
sunw       0.001880
left       0.001790
luggag     0.001745
last       0.001745
year       0.001704
movi       0.001670
westjet    0.001670
leav       0.001666
vancouv    0.001625
Name: CAN Word Counts, dtype: float64

In [33]:
top_ger_word_freq

seat         0.014133
servic       0.009921
good         0.009060
time         0.009020
crew         0.006888
class        0.005881
friendli     0.005232
one          0.004676
drink        0.004623
economi      0.004450
new          0.003576
nice         0.003470
even         0.003470
offer        0.003378
serv         0.003351
like         0.002596
old          0.002543
better       0.002305
also         0.002199
lh           0.002185
clean        0.002185
lufthansa    0.002185
choic        0.002172
well         0.002172
quit         0.002119
long         0.002027
small        0.002000
frankfurt    0.001987
free         0.001960
due          0.001934
realli       0.001907
price        0.001868
qualiti      0.001815
full         0.001815
space        0.001801
ife          0.001775
ok           0.001762
berlin       0.001669
snack        0.001669
work         0.001669
alway        0.001603
short        0.001576
inform       0.001563
ground       0.001550
system       0.001550
row       

#### Words that stand out for the each country:
US - delay, board, bag, gate, connect, ticket, cancel, late
(Not shockingly, Americans seem to have the least expansive vocabulary out of the top 5 countries. They also have the most usage of the most common words. And Americans seem to be most focus on the logistics of air travel) <br>
UK - cabin, leg <br>
AUS - food, meal, comfort, entertain 
(australians are hedonist and also probably because they fly so much focus on the experience) <br>
CAN - luggage, movie 
(most of the words are canadian related lol. would be interested to see if canadians leave the most positive reviews) <br>
GER - service, crew, friendly, drink, serv, drink, economic, clean snack, 
(Germans are mostly interested in service)

In [34]:
#compare top words based on cabin flown?? same process

**COSINE SIMILARITY- SPIRIT AIRLINES and BRITISH AIRWAYS**

In [58]:
df_spirit=airline_data[airline_data['airline_name']=='spirit-airlines']
df_spirit=df_spirit[['airline_name','content']] #limit to 15 rows to test

In [59]:
#list of texts for the content
text_list_spirit=list(df_spirit['content'])
text_list_spirit[0]

u"I was curious and nervous to try this airline due to other reviews I had read online but glad I did. While Spirit Airlines doesn't offer the inflight amenities that other airlines do, my flight was excellent. The flight boarded on time, went smoothly and arrived at my destination on top. Since I was tired, I appreciated the opportunity to rest and not be bothered. Others complain about the stringent rules Spirit Airlines has regarding carry-on luggage. My advice either pack light or follow the airline rules."

In [64]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)

cosine_total=0
count=len(text_list_spirit)
for text in text_list_spirit:
    text1 = "Ultra low fares Easy online booking and check-in Reliable, on-time service Clean, fuel efficient airplanes \
    Friendly staff Deluxe leather seating One personal item that fits under the seat Largest ULCC network in the U.S., Latin America and Caribbean\
    committed to offering the lowest total price to the places we fly, on average much lower than other airlines.\
    We help people save money and travel more often, create new jobs and stimulate business growth in the communities we serve"
    text2 = text

    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)

    cosine = get_cosine(vector1, vector2)
    cosine_total+=cosine
cosine_avg_spirit=float(cosine_total)/count    

print 'Cosine:', cosine_avg_spirit

Cosine: 0.339100343538


In [72]:
df_british=airline_data[airline_data['airline_name']=='british-airways']
df_british=df_british[['airline_name','content']]
#list of texts for the content
text_list_british=list(df_british['content'])

In [76]:
cosine_total=0
count=len(text_list_british)
for text in text_list_british:
    text1 = "Our customers will recognise that the service we offer is worth paying that little bit more \
    world’s leading global premium airline\
    forchoice for longhaul premium customers Deliver an outstanding service for customers \
    Grow our presence in key global cities leading position in London offer customers great value \
    bringing people together"
    text2 = text

    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)

    cosine = get_cosine(vector1, vector2)
    cosine_total+=cosine
cosine_avg_british=float(cosine_total)/count    

print 'Cosine:', cosine_avg_british

Cosine: 0.124313185049


**SENTIMENT ANALYSIS WORK**

In [90]:
#att dictionary
attr_dict = {'delayed':'scheduling','on time':'scheduling','late':'scheduling','early':'scheduling',
            'food':'service','flight attendant':'service','beverage':'service','drink':'service','breakfast':'service','lunch':'service','dinner':'service','meal':'service',
            'available':'booking','cancel':'booking','refund':'booking','ticket':'booking','customer service':'booking',
            'expensive':'price','cheap':'price','cost':'price','value':'price',
            'seat':'comfort','leg room':'comfort',
            'movie':'entertainment','tv':'entertainment','wifi':'entertainment','gogo':'entertainment'}
list_imp_words=attr_dict.keys()
for word in ['scheduling','service','booking','price','comfort','entertainment']:
    list_imp_words.append(word)
list_imp_words[0]

'refund'

In [105]:
df_spirit=airline_data[airline_data['airline_name']=='spirit-airlines']

#range(len(df_spirit['content']))

,airline_name,link,title,author,author_country,date,content,aircraft,type_traveller,cabin_flown,...,overall_rating,seat_comfort_rating,cabin_staff_rating,food_beverages_rating,inflight_entertainment_rating,ground_service_rating,wifi_connectivity_rating,value_money_rating,recommended,content_clean
31938,spirit-airlines,/airline-reviews/spirit-airlines,Spirit Airlines customer review,L Richardson,United States,2015-08-02,I was curious and nervous to try this airline ...,NaN,Solo Leisure,Economy,...,8.0,4.0,4.0,NaN,NaN,3.0,NaN,5.0,1,curious nervous try airline due reviews read o...
31950,spirit-airlines,/airline-reviews/spirit-airlines,Spirit Airlines customer review,Aaron Wortham,United States,2015-07-29,I'm extremely unimpressed with Spirit Airlines...,NaN,Couple Leisure,Economy,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0,i'm extremely unimpressed spirit airlines. und...
31951,spirit-airlines,/airline-reviews/spirit-airlines,Spirit Airlines customer review,F Soto,United States,2015-07-28,"Just because they advertise a low airfare, doe...",NaN,Solo Leisure,Economy,...,1.0,1.0,2.0,NaN,NaN,1.0,NaN,1.0,0,"advertise low airfare, doesn't mean you're get..."
31962,spirit-airlines,/airline-reviews/spirit-airlines,Spirit Airlines customer review,Vince Peterson,United States,2015-07-28,Any flyer who thinks they can save money with ...,NaN,FamilyLeisure,Economy,...,1.0,1.0,1.0,NaN,NaN,1.0,NaN,1.0,0,flyer thinks save money airline faced rude awa...
31963,spirit-airlines,/airline-reviews/spirit-airlines,Spirit Airlines customer review,Donna Zaccagnini,United States,2015-07-27,The plane was dirty. Empty potato chip wrapper...,NaN,FamilyLeisure,Economy,...,1.0,1.0,1.0,NaN,NaN,1.0,NaN,1.0,0,plane dirty. empty potato chip wrappers candy ...
31964,spirit-airlines,/airline-reviews/spirit-airlines,Spirit Airlines customer review,Nikalas Brasher,United States,2015-07-27,Horrible customer service. First there was a 2...,A320,Business,Economy,...,1.0,1.0,1.0,1.0,NaN,1.0,NaN,1.0,0,horrible customer service. first 2 hour wait t...
31975,spirit-airlines,/airline-reviews/spirit-airlines,Spirit Airlines customer review,Doan Vo,United States,2015-07-27,Cons: Did not have Pre-TSA available. Limited ...,NaN,Solo Leisure,Economy,...,1.0,3.0,3.0,NaN,NaN,4.0,NaN,2.0,0,cons: pre-tsa available. limited flights per d...
31976,spirit-airlines,/airline-reviews/spirit-airlines,Spirit Airlines customer review,Joseph Manenti,United States,2015-07-27,We thought that searching we would find a reas...,NaN,Couple Leisure,Economy,...,1.0,1.0,2.0,1.0,NaN,1.0,NaN,1.0,0,thought searching would find reasonable rate. ...
31988,spirit-airlines,/airline-reviews/spirit-airlines,Spirit Airlines customer review,Rajat Sharma,Canada,2015-07-27,"Well, (on July 22, 2015) like everyone else I ...",NaN,Solo Leisure,Economy,...,10.0,4.0,5.0,4.0,NaN,5.0,NaN,5.0,1,"well, (on july 22, 2015) like everyone else go..."
31989,spirit-airlines,/airline-reviews/spirit-airlines,Spirit Airlines customer review,S Griesemer,United States,2015-07-25,"The ""lower fee"" quickly surpasses competitors ...",A320,FamilyLeisure,Economy,...,3.0,1.0,3.0,1.0,1.0,1.0,1.0,2.0,0,"""lower fee"" quickly surpasses competitors chec..."


In [119]:
list_of_review_words=content_clean_words(df_spirit) #list of list of words
list_of_review_words[0] #remove punctuation later
#x = [''.join(c for c in s if c not in string.punctuation) for s in x]

[u'curious',
 u'nervous',
 u'try',
 u'airline',
 u'due',
 u'reviews',
 u'read',
 u'online',
 u'glad',
 u'did',
 u'.',
 u'spirit',
 u'airlines',
 u'does',
 u"n't",
 u'offer',
 u'inflight',
 u'amenities',
 u'airlines',
 u'do',
 u',',
 u'flight',
 u'excellent',
 u'.',
 u'flight',
 u'boarded',
 u'time',
 u',',
 u'went',
 u'smoothly',
 u'arrived',
 u'destination',
 u'top',
 u'.',
 u'since',
 u'tired',
 u',',
 u'appreciated',
 u'opportunity',
 u'rest',
 u'bothered',
 u'.',
 u'others',
 u'complain',
 u'stringent',
 u'rules',
 u'spirit',
 u'airlines',
 u'regarding',
 u'carry-on',
 u'luggage',
 u'.',
 u'advice',
 u'either',
 u'pack',
 u'light',
 u'follow',
 u'airline',
 u'rules',
 u'.']

In [136]:
df=pd.DataFrame(columns=['imp words','nearby words', 'sent score'])

nearby_words=[]
df_words=[]
for review in list_of_review_words:
    for word in list_imp_words:
            
        for i in range(len(review)):
            if word == review[i]:
                if i==0:
                    nearby_word_list = review[i] + ' ' + review[i+1] + ' ' + review[i+2]
                    nearby_words.append(nearby_word_list)
                    df_words.append(word)
                elif i==1:
                    nearby_word_list = review[i-1] + ' ' + review[i] + ' ' + review[i+1] + ' ' + review[i+2]
                    nearby_words.append(nearby_word_list)
                    df_words.append(word)
                elif i==(len(review)-2):
                    nearby_word_list = review[i-2] + ' ' + review[i-1] + ' ' + review[i] + ' ' + review[i+1]
                    nearby_words.append(nearby_word_list)
                    df_words.append(word)
                elif i==(len(review)-1):
                    nearby_word_list = review[i-2] + ' ' + review[i-1] + ' ' + review[i] 
                    nearby_words.append(nearby_word_list)
                    df_words.append(word)
                else:
                    nearby_word_list = review[i-2] + ' ' + review[i-1] + ' ' + review[i] + ' ' + review[i+1] + ' ' + review[i+2] 
                    nearby_words.append(nearby_word_list)
                    df_words.append(word)
                
nearby_words
df['imp words']=df_words
df['nearby words']=nearby_words
df[:2]

,imp words,nearby words,sent score
0,seat,"want random seat , drinks",NaN
1,cost,trying low cost budget airline,NaN


In [143]:
sid = SentimentIntensityAnalyzer()

for i in range(len(df)):
    
    temp_line = df['nearby words'][i]
    
    ss = sid.polarity_scores(temp_line)

    df['sent score'][i] = ss['compound']
    
df

,imp words,nearby words,sent score
0,seat,"want random seat , drinks",0.0772
1,cost,trying low cost budget airline,-0.2732
2,cost,. combine cost fact cancel,-0.25
3,cancel,cost fact cancel one connecting,-0.25
4,expensive,", makes expensive airline is",0
5,late,"whole day late , top",0.2023
6,seat,bag . seat requests anywhere,0
7,ticket,hour wait ticket counter causing,0
8,ticket,hour wait ticket counter got,0
9,service,horrible customer service . first,-0.5423


In [166]:
attr_dict = {'delayed':'scheduling','on time':'scheduling','late':'scheduling','early':'scheduling',
            'food':'service','flight attendant':'service','beverage':'service','drink':'service','breakfast':'service','lunch':'service','dinner':'service','meal':'service',
            'available':'booking','cancel':'booking','refund':'booking','ticket':'booking','customer service':'booking',
            'expensive':'price','cheap':'price','cost':'price','value':'price',
            'seat':'comfort','leg room':'comfort',
            'movie':'entertainment','tv':'entertainment','wifi':'entertainment','gogo':'entertainment'}

def replace_imp_words(s):
    attr=[]
    for word in s:
        if word in attr_dict:
            attr.append(attr_dict[word])
        else:
            attr.append(word)
    return attr
            
df['imp words2']=replace_imp_words(df['imp words'])

df['sent score']=df['sent score'].astype(float)
df.groupby('imp words2')[['sent score']].mean()

,sent score
imp words2,
booking,0.005273
comfort,0.043779
entertainment,0.264377
price,0.043566
scheduling,-0.138305
service,-0.039023


In [ ]:
words = ['passengers', 'crews']
dictionary = {}

for word in words:
    for i in range(len(line)):
        if word == line[i]:
            temp_list = line[i-2] + ' ' + line[i-1] + ' ' + line[i+1] + ' ' + line[i+2]
            x = {word:temp_list}
            dictionary.update(x)

dictionary

df = pd.DataFrame()
df['word'] = dictionary.keys()
df['string'] = dictionary.values()
# df['sent_score'] = ''
df['compound']=''
df

In [ ]:
for i in range(len(df)):
    
    temp_line = df['string'][i]
    
    ss = sid.polarity_scores(temp_line)

    df['compound'][i] = ss['compound']
    
#     df['compound'][i] = df['sent_score'][i]['compound']
    
df